In [144]:
# using SQLalchemy 
from sqlalchemy import create_engine
import pandas as pd
import pyodbc
import pandas as pd
import numpy as np

database = 'hackathon_danone'
username = 'hack12'
password = 'boA8AKUb54!'
driver= '{SQL Server}'
server = '192.168.250.3'

my_engine = create_engine(f"""mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver={server}', fast_executemany=True""" )

def db_sql(sql):
    cnxn = pyodbc.connect(f'DRIVER={driver};PORT=1433;SERVER={server};DATABASE={database};UID={username};PWD={password}')
    data = pd.read_sql(sql,cnxn)
    cnxn.close()    
    return data

In [145]:
import datetime

In [146]:
raw_mat_in = db_sql('select * from recipe_1_raw_material_in')
raw_mat_used = db_sql('select * from recipe_1_raw_material_used')

In [148]:
raw_bigbag = db_sql('select * from recipe_1_processing_details_bigbag')
raw_dd = db_sql('select * from recipe_1_processing_details_dd')
raw_slurry = db_sql('select * from recipe_1_processing_details_slurry')

In [149]:
raw_mat = pd.read_csv('raw_weighted_recipe1.csv', sep=',')

In [ ]:
walec = db_sql('select * from "Walec DD08" where timestamp between \'2018-06-03 19:00:00\' and \'2019-03-02 20:10:00\'')

In [150]:
raw_mat = raw_mat.loc[:, ~raw_mat.columns.isin(['Unnamed: 0'])]
raw_mat.head()

,id,material_code_2,process_order_sap,usage_pct,particles_grp1,particles_grp2,particles_grp3,moisture
0,0,30402480,5406843977,0.692825,0.547332,0.145493,0.0,8.244617
1,1,30402480,5407623983,0.517064,0.408481,0.108583,0.0,6.113119
2,2,10134160,1803061329,1.000000,0.790000,0.210000,0.0,11.800000
3,3,20268320,3606272673,0.914050,0.544255,0.369795,0.0,11.606613
4,4,30402480,5409354013,0.430278,0.272786,0.157492,0.0,5.563708


In [151]:
raw_out.head()

,orders_details_id,bigbag_number,bigbag_filling_time_end,bigbag_filling_duration,bigbag_weight,efficiency,class
0,0,1,2018-03-21 06:10:00,50.0,409.0,81.0,below
1,0,2,2018-03-21 07:00:00,50.0,402.0,80.0,below
2,0,3,2018-03-21 07:50:00,50.0,402.0,80.0,below
3,0,4,2018-03-21 08:40:00,50.0,411.0,82.0,below
4,0,5,2018-03-21 09:30:00,50.0,398.0,79.0,below


In [152]:
raw_bigbag.head()

,orders_details_id,bigbag_number,bigbag_filling_time_end,sifter_speed_nominal_pct
0,0,1,2018-03-21 06:10:00,1.0
1,0,2,2018-03-21 07:00:00,1.0
2,0,3,2018-03-21 07:50:00,1.0
3,0,4,2018-03-21 08:40:00,1.0
4,0,5,2018-03-21 09:25:00,1.0


In [153]:
out_bb = raw_out.merge(raw_bigbag, on=['orders_details_id', 'bigbag_number'])
out_bb.head()

,orders_details_id,bigbag_number,bigbag_filling_time_end_x,bigbag_filling_duration,bigbag_weight,efficiency,class,bigbag_filling_time_end_y,sifter_speed_nominal_pct
0,0,1,2018-03-21 06:10:00,50.0,409.0,81.0,below,2018-03-21 06:10:00,1.0
1,0,2,2018-03-21 07:00:00,50.0,402.0,80.0,below,2018-03-21 07:00:00,1.0
2,0,3,2018-03-21 07:50:00,50.0,402.0,80.0,below,2018-03-21 07:50:00,1.0
3,0,4,2018-03-21 08:40:00,50.0,411.0,82.0,below,2018-03-21 08:40:00,1.0
4,0,5,2018-03-21 09:30:00,50.0,398.0,79.0,below,2018-03-21 09:25:00,1.0


In [154]:
raw_bigbag['start_bigbag'] = raw_bigbag.apply(lambda x: x['bigbag_filling_time_end'] - datetime.timedelta(minutes=50), axis=1)

In [155]:
out_dd = pd.merge_asof(raw_bigbag.sort_values('start_bigbag'), raw_dd.sort_values('testing_time'),
                      left_on='start_bigbag', right_on='testing_time')
out_dd = out_dd.dropna()
out_dd.head(5)

,orders_details_id_x,bigbag_number,bigbag_filling_time_end,sifter_speed_nominal_pct,start_bigbag,orders_details_id_y,testing_time,steam_preasure,dd_speed,temp_out
2,0,3,2018-03-21 07:50:00,1.0,2018-03-21 07:00:00,0.0,2018-03-21 06:30:00,6.4,10.0,125.0
3,0,4,2018-03-21 08:40:00,1.0,2018-03-21 07:50:00,0.0,2018-03-21 07:30:00,6.4,10.0,125.0
4,0,5,2018-03-21 09:25:00,1.0,2018-03-21 08:35:00,0.0,2018-03-21 08:30:00,6.4,10.0,125.0
5,0,6,2018-03-21 10:20:00,1.0,2018-03-21 09:30:00,0.0,2018-03-21 09:30:00,6.4,10.0,125.0
6,0,7,2018-03-21 11:05:00,1.0,2018-03-21 10:15:00,0.0,2018-03-21 09:30:00,6.4,10.0,125.0


In [156]:
out_sl = pd.merge_asof(out_dd.sort_values('testing_time'), raw_slurry.sort_values('slurry_start_time'),
                      left_on='testing_time', right_on='slurry_start_time')

In [157]:
out_sl.head()

,orders_details_id_x,bigbag_number,bigbag_filling_time_end,sifter_speed_nominal_pct,start_bigbag,orders_details_id_y,testing_time,steam_preasure,dd_speed,temp_out,orders_details_id,slurry_process_order,slurry_line,slurry_start_time,water_pct,water_correction
0,0,3,2018-03-21 07:50:00,1.0,2018-03-21 07:00:00,0.0,2018-03-21 06:30:00,6.4,10.0,125.0,0,1,SL 2,2018-03-21 06:00:00,0.66,-40.0
1,0,4,2018-03-21 08:40:00,1.0,2018-03-21 07:50:00,0.0,2018-03-21 07:30:00,6.4,10.0,125.0,0,1,SL 2,2018-03-21 06:00:00,0.66,-40.0
2,0,5,2018-03-21 09:25:00,1.0,2018-03-21 08:35:00,0.0,2018-03-21 08:30:00,6.4,10.0,125.0,0,1,SL 2,2018-03-21 06:00:00,0.66,-40.0
3,0,6,2018-03-21 10:20:00,1.0,2018-03-21 09:30:00,0.0,2018-03-21 09:30:00,6.4,10.0,125.0,0,1,SL 2,2018-03-21 06:00:00,0.66,-40.0
4,0,7,2018-03-21 11:05:00,1.0,2018-03-21 10:15:00,0.0,2018-03-21 09:30:00,6.4,10.0,125.0,0,1,SL 2,2018-03-21 06:00:00,0.66,-40.0


In [158]:
datas = []
for i in range(len(out_sl)):
    start = out_sl['start_bigbag'][i].strftime('%Y-%m-%d %H:%M:%S')
    end = out_sl['bigbag_filling_time_end'][i].strftime('%Y-%m-%d %H:%M:%S')
    data = db_sql(f"""
    SELECT
        AVG(w.steam_pressure_at_the_inlet_of_regulation_unit),
        MIN(w.steam_pressure_at_the_inlet_of_regulation_unit),
        MAX(w.steam_pressure_at_the_inlet_of_regulation_unit),
        AVG(w.product_temperature_at_the_outlet_of_JetCooker),
        MIN(w.product_temperature_at_the_outlet_of_JetCooker),
        MAX(w.product_temperature_at_the_outlet_of_JetCooker),
        AVG(w.product_temperature_at_the_inlet),
        MIN(w.product_temperature_at_the_inlet),
        MAX(w.product_temperature_at_the_inlet),
        AVG(w.steam_pressure_at_the_inlet_of_JetCooker),
        MIN(w.steam_pressure_at_the_inlet_of_JetCooker),
        MAX(w.steam_pressure_at_the_inlet_of_JetCooker),
        AVG(w.condensate_temperature_at_DD_outlet),
        MIN(w.condensate_temperature_at_DD_outlet),
        MAX(w.condensate_temperature_at_DD_outlet)
    FROM
        "Walec DD08" as w
    WHERE
        timestamp between '{start}' and '{end}'""")
    datas.append(data)

In [159]:
vars = ['steam_pres', 'prod_temp_jetcooker','prod_temp', 'steam_pres_jetcooker', 'cond_temp']
aggs = ['avg', 'min', 'max']

In [160]:
from itertools import product

In [161]:
cols_avg = ["_".join(col) for col in list(product(vars, aggs))]

In [162]:
avg = pd.concat([data for data in datas], axis=0)
avg.columns = cols_avg
avg.head()


,steam_pres_avg,steam_pres_min,steam_pres_max,prod_temp_jetcooker_avg,prod_temp_jetcooker_min,prod_temp_jetcooker_max,prod_temp_avg,prod_temp_min,prod_temp_max,steam_pres_jetcooker_avg,steam_pres_jetcooker_min,steam_pres_jetcooker_max,cond_temp_avg,cond_temp_min,cond_temp_max
0,6.500740,6.481830,6.519011,110.807465,110.261765,111.303398,112.818550,112.496948,113.408379,18.96187,18.96187,18.96187,162.006411,161.743301,162.177322
0,6.498376,6.480447,6.518720,110.856509,110.370270,111.260002,112.597781,111.997826,113.169670,18.96187,18.96187,18.96187,162.031680,161.714371,162.177322
0,6.501342,6.446314,6.562603,111.317868,107.918076,115.708664,112.681835,108.721008,116.598396,18.96187,18.96187,18.96187,162.078167,161.772232,162.698135
0,6.499372,6.473993,6.525566,111.771414,108.286987,117.184319,113.997554,109.068214,116.316284,18.96187,18.96187,18.96187,162.068909,161.830109,162.350922
0,6.501587,6.479579,6.520236,113.350292,112.605446,113.755592,115.262928,114.601921,115.773766,18.96187,18.96187,18.96187,162.123015,161.974777,162.437729


In [ ]:
out_bb.shape

In [ ]:
avg_10.head()

In [ ]:
out_bb.shape

In [163]:
avg = avg.reset_index(drop=True)

In [128]:
avg.shape

(624, 15)

In [164]:
out_avg = pd.concat([out_sl, avg], axis=1)
out_avg.head()
out_avg[out_avg['orders_details_id_x'] == 11]

,orders_details_id_x,bigbag_number,bigbag_filling_time_end,sifter_speed_nominal_pct,start_bigbag,orders_details_id_y,testing_time,steam_preasure,dd_speed,temp_out,...,prod_temp_jetcooker_max,prod_temp_avg,prod_temp_min,prod_temp_max,steam_pres_jetcooker_avg,steam_pres_jetcooker_min,steam_pres_jetcooker_max,cond_temp_avg,cond_temp_min,cond_temp_max
578,11,1,2018-08-05 02:00:00,1.0,2018-08-05 01:10:00,24.0,2018-07-17 13:30:00.097,7.7,10.0,125.0,...,20.463854,21.416664,21.179981,21.592295,18.96187,18.96187,18.96187,20.645563,20.601294,20.803835
579,11,2,2018-08-05 03:05:00,1.0,2018-08-05 02:15:00,11.0,2018-08-05 01:30:00.000,6.0,8.0,121.0,...,20.485554,21.467082,21.375288,21.548893,18.96187,18.96187,18.96187,20.623380,20.572359,20.688097
580,11,3,2018-08-05 05:40:00,1.0,2018-08-05 04:50:00,11.0,2018-08-05 02:30:00.000,6.0,8.0,121.0,...,20.463854,21.456376,21.201681,21.635696,18.96187,18.96187,18.96187,20.481794,20.398752,20.572359
581,11,4,2018-08-05 06:25:00,1.0,2018-08-05 05:35:00,11.0,2018-08-05 05:30:00.000,6.0,8.0,121.0,...,20.116642,21.256222,21.093178,21.418690,18.96187,18.96187,18.96187,20.385828,20.196211,20.485556
582,11,5,2018-08-05 06:55:00,1.0,2018-08-05 06:05:00,11.0,2018-08-05 05:30:00.000,6.0,8.0,121.0,...,20.160042,21.240743,21.093178,21.418690,18.96187,18.96187,18.96187,20.372229,20.196211,20.485556
583,11,6,2018-08-05 07:45:00,1.0,2018-08-05 06:55:00,11.0,2018-08-05 06:30:00.000,6.3,10.0,121.0,...,20.420452,21.392576,21.201681,21.548893,18.96187,18.96187,18.96187,20.241445,20.167276,20.369818
584,11,7,2018-08-05 08:35:00,1.0,2018-08-05 07:45:00,11.0,2018-08-05 07:30:00.000,6.3,10.0,121.0,...,20.897869,21.566038,21.288485,21.852705,18.96187,18.96187,18.96187,20.283497,20.167276,20.427687
585,11,8,2018-08-05 09:10:00,1.0,2018-08-05 08:20:00,11.0,2018-08-05 07:30:00.000,6.3,10.0,121.0,...,20.550657,21.484732,21.201681,21.722500,18.96187,18.96187,18.96187,20.315421,20.196211,20.427687
586,11,9,2018-08-05 09:55:00,1.0,2018-08-05 09:05:00,11.0,2018-08-05 08:30:00.000,6.3,10.0,121.0,...,20.637461,21.501369,21.201681,21.722500,18.96187,18.96187,18.96187,20.334421,20.254080,20.514490
587,11,10,2018-08-05 11:00:00,1.0,2018-08-05 10:10:00,11.0,2018-08-05 09:30:00.000,6.3,10.0,121.0,...,20.637461,21.524734,21.245083,21.722500,18.96187,18.96187,18.96187,20.580943,20.427687,20.717031


In [ ]:
out_sl = out_sl.iloc[:, 1:]

In [165]:
out1 = out_avg.merge(raw_mat, left_on='orders_details_id_x', right_on='id')
out1.head()

,orders_details_id_x,bigbag_number,bigbag_filling_time_end,sifter_speed_nominal_pct,start_bigbag,orders_details_id_y,testing_time,steam_preasure,dd_speed,temp_out,...,cond_temp_min,cond_temp_max,id,material_code_2,process_order_sap,usage_pct,particles_grp1,particles_grp2,particles_grp3,moisture
0,0,3,2018-03-21 07:50:00,1.0,2018-03-21 07:00:00,0.0,2018-03-21 06:30:00,6.4,10.0,125.0,...,161.743301,162.177322,0,30402480,5406843977,0.692825,0.547332,0.145493,0.0,8.244617
1,0,4,2018-03-21 08:40:00,1.0,2018-03-21 07:50:00,0.0,2018-03-21 07:30:00,6.4,10.0,125.0,...,161.714371,162.177322,0,30402480,5406843977,0.692825,0.547332,0.145493,0.0,8.244617
2,0,5,2018-03-21 09:25:00,1.0,2018-03-21 08:35:00,0.0,2018-03-21 08:30:00,6.4,10.0,125.0,...,161.772232,162.698135,0,30402480,5406843977,0.692825,0.547332,0.145493,0.0,8.244617
3,0,6,2018-03-21 10:20:00,1.0,2018-03-21 09:30:00,0.0,2018-03-21 09:30:00,6.4,10.0,125.0,...,161.830109,162.350922,0,30402480,5406843977,0.692825,0.547332,0.145493,0.0,8.244617
4,0,7,2018-03-21 11:05:00,1.0,2018-03-21 10:15:00,0.0,2018-03-21 09:30:00,6.4,10.0,125.0,...,161.974777,162.437729,0,30402480,5406843977,0.692825,0.547332,0.145493,0.0,8.244617


In [166]:
val_recipe1_class = out1[out1['orders_details_id_x'].isin([11, 22, 23, 24])]

In [167]:
val_recipe1_class.to_csv('val_rec1.csv')

In [ ]:
raw_slurry.head(50)

In [ ]:
len(out1)

In [53]:
train = out1[out1['orders_details_id_x'] < 25]
test = out1[out1['orders_details_id_x'] >= 25]

In [54]:
out1.columns

Index(['orders_details_id_x', 'bigbag_number', 'bigbag_filling_time_end_x',
       'bigbag_filling_duration', 'bigbag_weight', 'efficiency', 'class',
       'bigbag_filling_time_end_y', 'sifter_speed_nominal_pct', 'start_bigbag',
       'orders_details_id_y', 'testing_time', 'steam_preasure', 'dd_speed',
       'temp_out', 'orders_details_id', 'slurry_process_order', 'slurry_line',
       'slurry_start_time', 'water_pct', 'water_correction', 'steam_pres_avg',
       'steam_pres_min', 'steam_pres_max', 'prod_temp_jetcooker_avg',
       'prod_temp_jetcooker_min', 'prod_temp_jetcooker_max', 'prod_temp_avg',
       'prod_temp_min', 'prod_temp_max', 'steam_pres_jetcooker_avg',
       'steam_pres_jetcooker_min', 'steam_pres_jetcooker_max', 'cond_temp_avg',
       'cond_temp_min', 'cond_temp_max', 'id', 'particles_grp1',
       'particles_grp2', 'particles_grp3', 'moisture', 'usage_pct', 'elems'],
      dtype='object')